# 🌐 WorldFlux Quick Start

**Unified interface for latent world models (DreamerV3, TD-MPC2)**

[![GitHub](https://img.shields.io/badge/GitHub-worldflux-blue?style=flat&logo=github)](https://github.com/worldflux/WorldFlux)
[![License: MIT](https://img.shields.io/badge/License-MIT-yellow.svg)](https://opensource.org/licenses/MIT)

This notebook demonstrates:
1. Installation & Setup
2. Creating World Models (one-liner API)
3. Encoding & Decoding Observations
4. Rollout API (the core feature!)
5. Simple Training Demo
6. Visualization

## 1. Installation

Install WorldFlux directly from GitHub:

In [ ]:
# Install WorldFlux with training dependencies
!pip install -q "worldflux[training,viz] @ git+https://github.com/worldflux/WorldFlux.git"

# Verify installation
import worldflux

print(f"WorldFlux version: {worldflux.__version__}")

In [ ]:
# Standard imports
import matplotlib.pyplot as plt
import torch

# WorldFlux imports
from worldflux import create_world_model, list_models

# Check GPU availability
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

## 2. Available Models

WorldFlux provides a unified interface for multiple world model architectures:

In [ ]:
# List all available model presets
models = list_models(verbose=True)

print("Available World Models:")
print("=" * 60)
for name, info in models.items():
    print(f"\n{name}")
    print(f"  Params: {info['params']}")
    print(f"  Type: {info['type']}")
    print(f"  Description: {info['description']}")

## 3. Create a World Model (One-Liner!)

The `create_world_model()` function provides a simple, unified interface:

In [ ]:
# Create a DreamerV3 model for image observations
# obs_shape: (channels, height, width) for images
dreamer = create_world_model(
    "dreamerv3:size12m",
    obs_shape=(3, 64, 64),  # RGB images
    action_dim=4,  # 4 discrete actions
    device=device,
)

print("DreamerV3 Model:")
print(f"  Parameters: {sum(p.numel() for p in dreamer.parameters()):,}")
print(f"  Obs shape: {dreamer.config.obs_shape}")
print(f"  Action dim: {dreamer.config.action_dim}")

In [ ]:
# Create a TD-MPC2 model for state observations
# Perfect for continuous control tasks like MuJoCo
tdmpc = create_world_model(
    "tdmpc2:5m",
    obs_shape=(39,),  # State vector (e.g., HalfCheetah has 39-dim state)
    action_dim=6,  # 6 continuous actions
    device=device,
)

print("TD-MPC2 Model:")
print(f"  Parameters: {sum(p.numel() for p in tdmpc.parameters()):,}")
print(f"  Obs shape: {tdmpc.config.obs_shape}")
print(f"  Action dim: {tdmpc.config.action_dim}")

## 4. Encode & Decode Observations

World models learn a compressed latent representation of observations:

In [ ]:
# Create a random observation (batch of 2 images)
batch_size = 2
obs = torch.randn(batch_size, 3, 64, 64, device=device)

print(f"Input observation shape: {obs.shape}")

# Encode: observation -> latent state
with torch.no_grad():
    state = dreamer.encode(obs)
    combined_features = torch.cat(
        [
            state.tensors["deter"],
            state.tensors["stoch"].flatten(1),
        ],
        dim=-1,
    )

print("")
print("Latent state components:")
print(f"  Deterministic (h): {state.tensors['deter'].shape}")
print(f"  Stochastic (z): {state.tensors['stoch'].shape}")
print(f"  Combined features: {combined_features.shape}")

In [ ]:
# Decode: latent state -> predictions
with torch.no_grad():
    output = dreamer.decode(state)

print("Decoded predictions:")
for key, value in output.preds.items():
    if isinstance(value, torch.Tensor):
        print(f"  {key}: {value.shape}")
    else:
        print(f"  {key}: {value}")

## 5. Rollout API 🚀

The core feature of world models: **roll out future states without environment interaction!**

This enables:
- Planning ahead without costly environment steps
- Learning from imagined experience
- Model-based RL algorithms

In [ ]:
# Start from an initial observation
initial_obs = torch.randn(1, 3, 64, 64, device=device)

# Encode to get initial state
with torch.no_grad():
    initial_state = dreamer.encode(initial_obs)
    initial_combined_features = torch.cat(
        [
            initial_state.tensors["deter"],
            initial_state.tensors["stoch"].flatten(1),
        ],
        dim=-1,
    )

# Define a sequence of actions (horizon=15 steps)
horizon = 15
action_dim = dreamer.config.action_dim

# Random action sequence: [horizon, batch_size, action_dim]
actions = torch.randn(horizon, 1, action_dim, device=device)

print(f"Initial state features: {initial_combined_features.shape}")
print(f"Action sequence shape: {actions.shape}")

In [ ]:
# Run imagination rollout!
with torch.no_grad():
    trajectory = dreamer.rollout(initial_state, actions)

print("Imagination Trajectory:")
print(f"  Number of states: {len(trajectory.states)}")
print(f"  Predicted rewards shape: {trajectory.rewards.shape}")
print(f"  Continue probabilities shape: {trajectory.continues.shape}")

# Access individual predicted states
features_t0 = torch.cat(
    [
        trajectory.states[0].tensors["deter"],
        trajectory.states[0].tensors["stoch"].flatten(1),
    ],
    dim=-1,
)
features_t14 = torch.cat(
    [
        trajectory.states[-1].tensors["deter"],
        trajectory.states[-1].tensors["stoch"].flatten(1),
    ],
    dim=-1,
)
print("")
print(f"  State at t=0: features {features_t0.shape}")
print(f"  State at t=14: features {features_t14.shape}")

In [ ]:
# Visualize predicted rewards over the imagination horizon
plt.figure(figsize=(10, 4))

rewards = trajectory.rewards.cpu().numpy().flatten()
continues = trajectory.continues.cpu().numpy().flatten()

plt.subplot(1, 2, 1)
plt.plot(rewards, marker="o", linewidth=2, markersize=4)
plt.xlabel("Time Step")
plt.ylabel("Predicted Reward")
plt.title("Imagined Rewards")
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(continues, marker="s", linewidth=2, markersize=4, color="green")
plt.xlabel("Time Step")
plt.ylabel("Continue Probability")
plt.title("Episode Continuation")
plt.grid(True, alpha=0.3)
plt.ylim([0, 1.1])

plt.tight_layout()
plt.show()

## 6. Step-by-Step Prediction

You can also make single-step predictions with `transition()` and `update()`:

In [ ]:
# transition(): Prior prediction (imagination, no observation)
action = torch.randn(1, action_dim, device=device)

with torch.no_grad():
    # Predict next state without seeing actual observation
    next_state_prior = dreamer.transition(initial_state, action)

print("Prior prediction (imagination):")
print(f"  Deterministic: {next_state_prior.tensors['deter'].shape}")
print(f"  Stochastic: {next_state_prior.tensors['stoch'].shape}")

In [ ]:
# update(): Posterior update (with actual observation)
next_obs = torch.randn(1, 3, 64, 64, device=device)  # Real observation

with torch.no_grad():
    # Update state with actual observation (posterior)
    next_state_posterior = dreamer.update(initial_state, action, next_obs)

print("Posterior update (with observation):")
print(f"  Deterministic: {next_state_posterior.tensors['deter'].shape}")
print(f"  Stochastic: {next_state_posterior.tensors['stoch'].shape}")

# The posterior is more accurate because it incorporates the actual observation
diff = (next_state_prior.tensors["stoch"] - next_state_posterior.tensors["stoch"]).abs().mean()
print("")
print(f"Prior vs Posterior stochastic difference: {diff:.4f}")

## 7. Training Demo

WorldFlux provides a complete training infrastructure:

In [ ]:
from worldflux.training import Trainer, TrainingConfig
from worldflux.training.data import create_random_buffer

# Create a fresh model for training
model = create_world_model(
    "dreamerv3:size12m",
    obs_shape=(3, 64, 64),
    action_dim=4,
    device=device,
)

# Create a replay buffer with random data for demo
# In practice, you'd collect real environment data
buffer = create_random_buffer(
    capacity=5000,
    obs_shape=(3, 64, 64),
    action_dim=4,
    num_episodes=50,
    episode_length=100,
    seed=42,
)

print(f"Buffer size: {len(buffer)} transitions")
print(f"Number of episodes: {buffer.num_episodes}")

In [ ]:
# Configure training (reduced for demo)
config = TrainingConfig(
    total_steps=100,  # Quick demo - use 50000+ for real training
    batch_size=8,  # Small batch for Colab
    sequence_length=16,  # Shorter sequences
    learning_rate=3e-4,
    log_interval=50,
)

# Create trainer
trainer = Trainer(model, config)

print("Training configuration:")
print(f"  Total steps: {config.total_steps}")
print(f"  Batch size: {config.batch_size}")
print(f"  Sequence length: {config.sequence_length}")

In [ ]:
# Train the model
print("Starting training...\n")
trained_model = trainer.train(buffer)

# Evaluate on a batch to show final metrics
model.eval()
with torch.no_grad():
    batch = buffer.sample(batch_size=8, seq_len=16, device=device)
    losses = model.loss(batch)

print("\nTraining complete!")
print("Final losses:")
print(f"  total: {losses.loss.item():.4f}")
for name, value in losses.components.items():
    print(f"  {name}: {value.item():.4f}")

In [ ]:
# Verify the trained model works
# Run a quick imagination rollout to confirm everything is working
test_obs = torch.randn(1, 3, 64, 64, device=device)
test_actions = torch.randn(5, 1, 4, device=device)

with torch.no_grad():
    state = model.encode(test_obs)
    combined_features = torch.cat(
        [
            state.tensors["deter"],
            state.tensors["stoch"].flatten(1),
        ],
        dim=-1,
    )
    trajectory = model.rollout(state, test_actions)

print("Model verification:")
print(f"  Encoded state features: {combined_features.shape}")
print(f"  Imagined trajectory length: {len(trajectory.states)}")
print(f"  Predicted rewards: {trajectory.rewards.squeeze().tolist()}")

## 8. Model Comparison: DreamerV3 vs TD-MPC2

| Feature | DreamerV3 | TD-MPC2 |
|---------|-----------|----------|
| **Input Type** | Images or State | State |
| **Latent Space** | Categorical (discrete) | SimNorm (continuous) |
| **Decoder** | Yes (reconstructs obs) | No (implicit model) |
| **Best For** | Atari, visual tasks | MuJoCo, robotics |
| **Planning** | Policy rollouts | MPC with Q-function |

In [ ]:
# Compare latent state structures
print("DreamerV3 State Structure:")
dreamer_obs = torch.randn(1, 3, 64, 64, device=device)
with torch.no_grad():
    dreamer_state = dreamer.encode(dreamer_obs)
print(f"  Deterministic (RSSM hidden): {dreamer_state.tensors['deter'].shape}")
print(f"  Stochastic (categorical): {dreamer_state.tensors['stoch'].shape}")
print("  Has decoder: Yes")

print("")
print("TD-MPC2 State Structure:")
tdmpc_obs = torch.randn(1, 39, device=device)
with torch.no_grad():
    tdmpc_state = tdmpc.encode(tdmpc_obs)
print(f"  Deterministic (SimNorm): {tdmpc_state.tensors['latent'].shape}")
print("  Has decoder: No (implicit model)")

## 9. Save & Load Models

In [ ]:
import os

# Save the trained model
save_path = "./my_world_model"
model.save_pretrained(save_path)
print(f"Model saved to: {save_path}")

# List saved files
print("\nSaved files:")
for f in os.listdir(save_path):
    print(f"  {f}")

In [ ]:
# Load the saved model
loaded_model = create_world_model(save_path, device=device)
print(f"Loaded model type: {type(loaded_model).__name__}")
print(f"Loaded model config: {loaded_model.config}")

## 10. Next Steps

Congratulations! You've learned the basics of WorldFlux. Here are some next steps:

### Learn More
- [Full Documentation](https://worldflux.readthedocs.io)
- [GitHub Repository](https://github.com/worldflux/WorldFlux)
- [Examples Directory](https://github.com/worldflux/WorldFlux/tree/main/examples)

### Try These
1. **Train on real data**: Collect trajectories from Atari or MuJoCo
2. **Use larger models**: Try `dreamerv3:size50m` or `tdmpc2:48m`
3. **Build an agent**: Use imagination for planning

### Example Commands
```bash
# Collect Atari data
python examples/collect_atari.py --env Breakout --episodes 100

# Train DreamerV3
python examples/train_atari_dreamer.py --data atari_data.npz --steps 100000

# Visualize imagination
python examples/visualize_imagination.py --model ./outputs/model
```

---

**Questions or Issues?** Open an issue on [GitHub](https://github.com/worldflux/WorldFlux/issues)

**Want to contribute?** Check out our [Contributing Guide](https://github.com/worldflux/WorldFlux/blob/main/CONTRIBUTING.md)